# Problem 01 - solution

The following are my approach to solving these problems.  Note that there may be more than one approach to each, and we could even debate the exact solutions.


## Problem 1 - Word Counts



### Part A - Part A. Characters in Little Women

#### How many times are each of the following characters mentioned by name in the text of Little Women: Jo, Beth, Meg, Amy

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/women.txt

--2016-09-26 13:37:51--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/women.txt
Resolving raw.githubusercontent.com... 151.101.32.133
Connecting to raw.githubusercontent.com|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1053440 (1.0M) [text/plain]
Saving to: ‘women.txt’

women.txt           100%[===================>]   1.00M  1.13MB/s    in 0.9s    

2016-09-26 13:37:53 (1.13 MB/s) - ‘women.txt’ saved [1053440/1053440]



In [21]:
!cat women.txt | grep -oE '\w{{2,}}' \
   | grep -e "Jo\|Beth\|Meg\|Amy" \
   | tr '[:upper:]' '[:lower:]' \
   | sort | uniq -c | sort -rn

1355 jo
 683 meg
 645 amy
 459 beth
 163 john
   6 joanna
   5 bethy
   4 josephine
   3 jove
   3 josy
   2 johnson
   1 megs
   1 meggy
   1 jouvin
   1 josyphine
   1 joneses
   1 jones
   1 joian
   1 johnsonianly
   1 johnsonian
   1 beths


As we can see from the output of the command above, by the straight wording of the question, there are exactly 1,355 mentions of Jo, 683 of Meg, 645 of Amy, and 459 of Beth in Little Women.  If we were to assume that diminutive or nickname forms might count as well, we might add mentions of "Megs", "Bethy", and "Meggy" to these counts, for example.  For the purposes of this solution, however, I assume that these are not required, because the text might need to be consulted directly by someone familiar with the novel to determine which nicknames are valid, which seems beyond the scope of the assignment.

### Part B - Juliet and Romeo in Romeo and Juliet

#### How many times do each of the characters Juliet and Romeo have speaking lines in Romeo and Juliet? Keep in mind that this is the text of a play.

In [22]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/romeo.txt

--2016-09-26 13:49:06--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/romeo.txt
Resolving raw.githubusercontent.com... 151.101.32.133
Connecting to raw.githubusercontent.com|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178983 (175K) [text/plain]
Saving to: ‘romeo.txt’

romeo.txt           100%[===================>] 174.79K   999KB/s    in 0.2s    

2016-09-26 13:49:07 (999 KB/s) - ‘romeo.txt’ saved [178983/178983]



First we must recall -- as the problem highlights -- that this text is that of a play.  Because of this, we cannot simply count mentions of "Romeo," as we might accidentally inflate the count due to mentions of this character, for example, by other characters in their speaking lines.  Instead, we must first look for a patter that indicates Romeo's speaking lines specifically.

In [33]:
!cat romeo.txt | grep "Rom" | head -25

In this brief sample, we can see title lines and metadata that include mention of Romeo, and both stage directions ("Enter Romeo") and spoken lines that include his name.  What stands out, though, is that lines spoken by Romeo appear to be delineated by "Rom.", so we can search for this specific pattern.  Let's verify that the same should hold true for mentions of Juliet.

In [34]:
!cat romeo.txt | grep "Jul" | head -25

We see that the pattern seems to hold for both.  I will assume that matches of the exact characters "Rom." and "Jul." indicate the start of a speaking line for one or the other characters, and will explicitly count only those lines.

In [40]:
!cat romeo.txt | grep -w "Rom\." \
   | grep -oE '\w{{2,}}\.' \
   | grep "Rom" \
   | sort | uniq -c | sort -rn

 163 Rom.


In [41]:
!cat romeo.txt | grep -w "Jul\." \
   | grep -oE '\w{{2,}}\.' \
   | grep "Jul" \
   | sort | uniq -c | sort -rn

 117 Jul.


The two pipelines above indicate that Romeo has 163 speaking lines, while Juliet has only 117.  To match the specific case with a trailing `.`, the first regular expressions in both above cases use the `-w` flag to denote a word match and the escape sequence `\.` to match the literal trailing period.  The second regular expressions include this literal at the end of the match sequence as well, with the trailing literal period in `'\w{{2,}}\.'` requiring that the match include the period at the end.

## Problem 2 - Capital Bikeshare

### Part A - Station counts

#### Which 10 Capital Bikeshare stations were the most popular departing stations in Q1 2016?

In [42]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/2016q1.csv.zip

--2016-09-26 14:02:12--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/2016q1.csv.zip
Resolving raw.githubusercontent.com... 151.101.32.133
Connecting to raw.githubusercontent.com|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10643003 (10M) [application/octet-stream]
Saving to: ‘2016q1.csv.zip’

2016q1.csv.zip      100%[===================>]  10.15M  1.18MB/s    in 8.8s    

2016-09-26 14:02:22 (1.15 MB/s) - ‘2016q1.csv.zip’ saved [10643003/10643003]



In [43]:
!unzip 2016q1.csv.zip

Archive:  2016q1.csv.zip
  inflating: 2016q1.csv              


In [47]:
!head -5 2016q1.csv | csvlook

|----------------+-----------------+---------------+----------------------+--------------------------------+--------------------+---------------------------+-------------+--------------|
|  Duration (ms) | Start date      | End date      | Start station number | Start station                  | End station number | End station               | Bike number | Member Type  |
|----------------+-----------------+---------------+----------------------+--------------------------------+--------------------+---------------------------+-------------+--------------|
|  301295        | 3/31/2016 23:59 | 4/1/2016 0:04 | 31280                | 11th & S St NW                 | 31506              | 1st & Rhode Island Ave NW | W00022      | Registered   |
|  557887        | 3/31/2016 23:59 | 4/1/2016 0:08 | 31275                | New Hampshire Ave & 24th St NW | 31114              | 18th St & Wyoming Ave NW  | W01294      | Registered   |
|  555944        | 3/31/2016 23:59 | 4/1/2016 0:08 | 31101       

In [55]:
!csvcut -n 2016q1.csv

  1: Duration (ms)
  2: Start date
  3: End date
  4: Start station number
  5: Start station
  6: End station number
  7: End station
  8: Bike number
  9: Member Type


In [58]:
!csvcut -c5 2016q1.csv | tail -n +2 | csvsort | uniq -c | sort -rn | head -10

13120 Columbus Circle / Union Station
9560 Massachusetts Ave & Dupont Circle NW
9388 Lincoln Memorial
8138 Jefferson Dr & 14th St SW
7479 Thomas Circle
7401 15th & P St NW
6568 14th & V St NW
6491 New Hampshire Ave & T St NW
5649 Eastern Market Metro / Pennsylvania Ave & 7th St SE
5514 17th & Corcoran St NW


As we can see in the above results, the top ten starting stations in this time period were led by Columbus Circle / Union Station with over 13,000 rides, followed by Dupont Circle and the Lincoln Memorial and the rest as listed.

In the pipeline above, `tail -n +2` ensures we skip the header line before the sort process begins.

#### Which 10 were the most popular destination stations in Q1 2016?

In [59]:
!csvcut -c7 2016q1.csv | tail -n +2 | csvsort | uniq -c | sort -rn | head -10

13880 Columbus Circle / Union Station
11183 Massachusetts Ave & Dupont Circle NW
9419 Lincoln Memorial
8975 Jefferson Dr & 14th St SW
8092 15th & P St NW
7267 14th & V St NW
6997 Thomas Circle
6245 New Hampshire Ave & T St NW
5761 5th & K St NW
5651 17th & Corcoran St NW


The above results show us very similar numbers for destination stations during the same time period, with the first four stations unchanged and led again by Union Station with over 13,000 rides.  Thomas Circle appears to be a more prominent start station than end station, as does Eastern Market, which does not even make the top ten destination stations.

### Part B - bike counts

#### For the most popular departure station, which 10 bikes were used most in trips departing from there?

In this part, we will use `csvgrep` to select only the required stations - Union Station, in both cases.

In [60]:
!csvgrep -c5 -m "Columbus Circle / Union Station" 2016q1.csv | head

Duration (ms),Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member Type
179833,3/31/2016 23:31,3/31/2016 23:34,31623,Columbus Circle / Union Station,31616,3rd & H St NE,W20445,Registered
701676,3/31/2016 23:13,3/31/2016 23:25,31623,Columbus Circle / Union Station,31601,19th & East Capitol St SE,W21148,Registered
709799,3/31/2016 23:01,3/31/2016 23:13,31623,Columbus Circle / Union Station,31611,13th & H St NE,W01261,Registered
274002,3/31/2016 22:46,3/31/2016 22:50,31623,Columbus Circle / Union Station,31603,1st & M St NE,W21049,Registered
533503,3/31/2016 22:38,3/31/2016 22:47,31623,Columbus Circle / Union Station,31622,13th & D St NE,W01253,Registered
465837,3/31/2016 21:36,3/31/2016 21:44,31623,Columbus Circle / Union Station,31640,Maryland Ave & E St NE,W00086,Registered
371081,3/31/2016 21:11,3/31/2016 21:18,31623,Columbus Circle / Union Station,31632,15th & F St NE,W00736,Registered
569451,3/31/2016 21:11,3/31/2016 21:21,31623,Co

We can further limit the columns used to cut down on the data flowing through the pipe.

In [61]:
!csvcut -c5,8 2016q1.csv | csvgrep -c1 -m "Columbus Circle / Union Station" | head

Start station,Bike number
Columbus Circle / Union Station,W20445
Columbus Circle / Union Station,W21148
Columbus Circle / Union Station,W01261
Columbus Circle / Union Station,W21049
Columbus Circle / Union Station,W01253
Columbus Circle / Union Station,W00086
Columbus Circle / Union Station,W00736
Columbus Circle / Union Station,W00945
Columbus Circle / Union Station,W21751


In [66]:
!csvcut -c5,8 2016q1.csv \
    | csvgrep -c1 -m "Columbus Circle / Union Station" \
    | csvcut -c2 \
    | tail -n +2 \
    | sort | uniq -c | sort -rn | head -12

  17 W22227
  16 W21867
  16 W21641
  16 W21538
  16 W21239
  16 W20540
  16 W00714
  15 W22080
  15 W21450
  15 W21076
  15 W00777
  15 W00288


Above are the most commonly used bikes in trips departing from Union Station, led by bike number W22227.  As we might expect it appears that the distribution seems rather uniform.  Note that because several bikes had exactly 15 trips starting from Union Station, the list includes the top twelve bikes, rather than the top ten.

#### Which 10 bikes were used most in trips ending at the most popular destination station?

In [68]:
!csvcut -c7,8 2016q1.csv \
    | csvgrep -c1 -m "Columbus Circle / Union Station" \
    | csvcut -c2 \
    | tail -n +2 \
    | sort | uniq -c | sort -rn | head -15

  18 W00485
  17 W22227
  16 W22099
  16 W22080
  16 W21239
  16 W21076
  16 W20425
  16 W00714
  15 W21997
  15 W21867
  15 W21641
  15 W21538
  15 W21450
  15 W20540
  15 W01439


Above are the most commonly used bikes in trips arriving at Union Station, let by bike number W00485.  It is interesting to note that bike W22227, the top departing bike, is in second place, but bike W00485, the top arriving bike, does not appear in the top ten departing bikes.  In any case these also seem at first glance to be uniformly distributed.  Again, the list is expanded, this time to fifteen bikes, to account for the tie at exactly fifteen trips.

## Problem 3 - Filters

### Part A - split and lowercase filters

Write a Python filter than replaces grep -oE '\w{2,}' to split lines of text into one word per line, and write an additional Python filter to replace tr '[:upper:]' '[:lower:]' to transform text into lower case.

With your two new filters, repeat the original pipeline, and substitute your new filters as appropriate. You should obtain the same results.

In [69]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/simplefilter.py

--2016-09-26 14:27:23--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/simplefilter.py
Resolving raw.githubusercontent.com... 151.101.32.133
Connecting to raw.githubusercontent.com|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208 [text/plain]
Saving to: ‘simplefilter.py’

simplefilter.py     100%[===================>]     208  --.-KB/s    in 0s      

2016-09-26 14:27:23 (19.8 MB/s) - ‘simplefilter.py’ saved [208/208]



In [70]:
!cp simplefilter.py split.py

The file `split.py` is modified from the template to split lines of text into one word per line.  To demonstrate this, we can compare the original pipeline with a new pipeline with `split.py` substituting for the first `grep` command.

In [77]:
!cat women.txt \
    | grep -oE '\w{{1,}}' \
    | tr '[:upper:]' '[:lower:]' \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -10

8155 and
7689 the
5152 to
4531 a
4003 i
3523 of
3245 her
2774 it
2503 in
2447 you
sort: write failed: standard output: Broken pipe
sort: write error


We can ignore the broken pipe and related errors as the output appears to be correct.

Next, we repeat the pipeline with `split.py` substituted:

In [74]:
!chmod +x split.py

Examining the filter script below, the key line, #14, removes trailing newlines, splits tokens by the space (`' '`), and removes words that are not entirely alphabetical.

In [83]:
!grep -n '' split.py

1:#!/usr/bin/env python
2:
3:"""
4:A filter that splits incoming lines of text into one word per line.
5:It eliminates all tokens that are non-alphabetic, containing numbers,
6:punctuation, or whitespace.
7:""" 
8:
9:import fileinput
10:
11:
12:def process(line):
13:    """For each line of input, split it into words."""
14:    for token in [w for w in line[:-1].split(' ') if w.isalpha()]:
15:        print(token)
16:
17:
18:for line in fileinput.input():
19:    process(line)


In [78]:
!cat women.txt \
    | ./split.py \
    | tr '[:upper:]' '[:lower:]' \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -10

7408 and
6963 the
4749 to
4064 a
3251 of
2626 her
2525 i
2275 in
2066 she
2057 for
sort: write failed: standard output: Broken pipe
sort: write error


Almost the exact words listed appear in nearly the same order, but with lower counts for each.  We can examine the output of each command to see if there are obvious differences:

In [79]:
!cat women.txt | grep -oE '\w{{2,}}' | head -25

The
Project
Gutenberg
EBook
of
Little
Women
by
Louisa
May
Alcott
This
eBook
is
for
the
use
of
anyone
anywhere
at
no
cost
and
with


In [80]:
!cat women.txt | ./split.py | head -25

Project
Gutenberg
EBook
of
Little
by
Louisa
May
This
eBook
is
for
the
use
of
anyone
anywhere
at
no
cost
and
almost
no
restrictions
You
Traceback (most recent call last):
  File "./split.py", line 19, in <module>
    process(line)
  File "./split.py", line 15, in process
    print(token)
BrokenPipeError: [Errno 32] Broken pipe
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe
cat: stdout: Broken pipe


We can see straight away on the first few lines that there is a difference.  Let's look at the text itself:

In [81]:
!head -3 women.txt

Three obvious issues jump out.  First, the initial "The" is elided; it is not clear why.  Next, "Women" is removed, perhaps due to the trailing comma, which will cause the token to fail the `isalpha()` test.  Also, "Alcott" is removed, perhaps having to do with its position at the end of the line.

We can update the filter to use Python's regular expression model and a similar expression, `\w{1,}` to find all matches more intelligently.  Here the regular expression is prepared in line 13 and used in line 18.

In [88]:
!grep -n '' split.py

1:#!/usr/bin/env python
2:
3:"""
4:A filter that splits incoming lines of text into one word per line.
5:It eliminates all tokens that are non-alphabetic, containing numbers,
6:punctuation, or whitespace.
7:""" 
8:
9:import fileinput
10:import re
11:
12:
13:RE_SPLIT = re.compile('(\w{1,})')
14:
15:
16:def process(line):
17:    """For each line of input, split it into words."""
18:    for token in [w for w in RE_SPLIT.findall(line)]:
19:        print(token)
20:
21:
22:for line in fileinput.input():
23:    process(line)


In [89]:
!cat women.txt | ./split.py | head -25

The
Project
Gutenberg
EBook
of
Little
Women
by
Louisa
May
Alcott
This
eBook
is
for
the
use
of
anyone
anywhere
at
no
cost
and
with
Traceback (most recent call last):
  File "./split.py", line 23, in <module>
    process(line)
  File "./split.py", line 19, in process
    print(token)
BrokenPipeError: [Errno 32] Broken pipe
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe
cat: stdout: Broken pipe


This looks much better.  We can try the full pipeline again:

In [90]:
!cat women.txt \
    | ./split.py \
    | tr '[:upper:]' '[:lower:]' \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -10

8155 and
7689 the
5152 to
4531 a
4003 i
3523 of
3245 her
2774 it
2503 in
2447 you
sort: write failed: standard output: Broken pipe
sort: write error


This looks to be an exact match.

In [71]:
!cp simplefilter.py lowercase.py

The filter `lowercase.py` is modified from the template to lowercase incoming lines of text.

In [91]:
!chmod +x lowercase.py

In [95]:
!grep -n '' lowercase.py

1:#!/usr/bin/env python
2:
3:"""
4:A filter that lower cases all incoming text.
5:"""
6:
7:import fileinput
8:
9:
10:def process(line):
11:    """For each line of input, lower case it and print the result."""
12:    print(line[:-1].lower())
13:
14:
15:for line in fileinput.input():
16:    process(line)


Note that the only line aside from the comments that changes in the above script is line #12, which adds the `lower()` to the print statement.

In [92]:
!head women.txt | ./lowercase.py

This looks correct, so we'll first attempt to replace the original pipeline's use of `tr` with `lowercase.py`:

In [93]:
!cat women.txt \
    | grep -oE '\w{{1,}}' \
    | ./lowercase.py \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -10

8155 and
7689 the
5152 to
4531 a
4003 i
3523 of
3245 her
2774 it
2503 in
2447 you
sort: write failed: standard output: Broken pipe
sort: write error


Looks good so far, we are seeing the exact same counts.  To address the problem's challenge, we finally replace both filters at once.

In [94]:
!cat women.txt \
    | ./split.py \
    | ./lowercase.py \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -10

8155 and
7689 the
5152 to
4531 a
4003 i
3523 of
3245 her
2774 it
2503 in
2447 you
sort: write failed: standard output: Broken pipe
sort: write error


This completes Problem 3 - Part A.

### Part B - stop words

#### Write a Python filter that removes at least ten common words of English text, commonly known as "stop words". Sources of English stop word lists are readily available online, or you may generate your own list from the text.

We begin by acquiring a common list of English stop words, gathered from the site http://www.textfixer.com/resources/common-english-words.txt as linked from the [Wikipedia page on stop words](https://en.wikipedia.org/wiki/Stop_words).

In [96]:
!wget http://www.textfixer.com/resources/common-english-words.txt

--2016-09-26 15:00:23--  http://www.textfixer.com/resources/common-english-words.txt
Resolving www.textfixer.com... 216.172.105.85
Connecting to www.textfixer.com|216.172.105.85|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 551 [text/plain]
Saving to: ‘common-english-words.txt’

common-english-word 100%[===================>]     551  --.-KB/s    in 0s      

2016-09-26 15:00:23 (7.73 MB/s) - ‘common-english-words.txt’ saved [551/551]



In [97]:
!head common-english-words.txt

a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your

Next we copy the template filter script as before, renaming it appropriately.

In [98]:
!cp simplefilter.py stopwords.py

In [99]:
!chmod +x stopwords.py

In [107]:
!grep -n '' stopwords.py

1:#!/usr/bin/env python
2:
3:"""
4:A filter that removes a set of stop words obtained from a local file.
5:
6:This filter assumes that incoming text has been prepared to arrive as
7:one word per line, in lower case.
8:"""
9:
10:import fileinput
11:
12:
13:STOPWORDS = open('common-english-words.txt', 'r').read().strip().split(',')
14:
15:
16:def process(line):
17:    """For each line of input, remove any matching stopwords."""
18:    # Remove trailing newline first
19:    line = line[:-1]
20:    if line not in STOPWORDS:
21:        print(line)
22:
23:    
24:for line in fileinput.input():
25:    process(line[:-1])


The key changes in `stopwords.py` from the template are line #13, which imports the list of stopwords, and line #20, which checks whether an incoming word is in the stopword list.  Note also that in line #19 the removal of a trailing newline occurs before checking for stopwords.

The assumption that incoming text will already be split into one word per line and lowercased is stated explicitly in the first comment, lines #6-7.

In [108]:
!head women.txt | ./split.py | ./lowercase.py | ./stopwords.py

project
gutenberg
ebook
little
women
louisa
alcott
ebook
use
anyone
anywhere
cost
restrictions
whatsoever
copy
give
away
re
use
under
terms
project
gutenberg
license
included
ebook
online
www
gutenberg
net
title
little
women


This appears to be correct.  Let's put it all together:

In [123]:
!cat women.txt \
    | ./split.py \
    | ./lowercase.py \
    | ./stopwords.py \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -25

1680 t
1495 s
1362 jo
 730 little
 725 one
 686 meg
 658 up
 652 amy
 598 laurie
 550 don
 492 very
 485 out
 467 beth
 462 good
 399 now
 398 m
 393 go
 380 old
 377 mother
 375 never
 374 much
 368 well
 361 ll
 360 see
 355 over
sort: write failed: standard output: Broken pipe
sort: write error


This would seem to be correct - we see the names we looked for earlier appearing near the top of the list, and common stop words are indeed removed - however the list starts with odd "words", in "t", "s", "m", and "ll".  Is it possible that these are occurences of contractions?  We can check a few different ways.  First, let's see if our `split.py` is causing the problem:

In [125]:
!cat women.txt \
    | grep -oE '\w{{1,}}' \
    | ./lowercase.py \
    | ./stopwords.py \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -25

1680 t
1495 s
1362 jo
 730 little
 725 one
 686 meg
 658 up
 652 amy
 598 laurie
 550 don
 492 very
 485 out
 467 beth
 462 good
 399 now
 398 m
 393 go
 380 old
 377 mother
 375 never
 374 much
 368 well
 361 ll
 360 see
 355 over
sort: write failed: standard output: Broken pipe
sort: write error


No, the results are exactly the same.  Instead, we'll need to look for occurrences of "t" and "s" by themselves.  The `--context` option to `grep` might help us here, pointing out surrounding text to search for in the source.

In [126]:
!cat women.txt \
    | ./split.py \
    | ./lowercase.py \
    | grep --context=2 -oE '^t$' \
    | head -20

christmas
won
t
be
christmas
--
--
i
don
t
think
it
--
--
we
haven
t
got
father
--


In [127]:
!grep -i "we haven't got" women.txt

Aha, it does appear that the occurences of a bare "t" are from contractions.  Let's repeat with "s", which might occur in possessives.

In [128]:
!cat women.txt \
    | ./split.py \
    | ./lowercase.py \
    | grep --context=2 -oE '^s$' \
    | head -20

seven
amy
s
valley
of
--
--
nineteen
amy
s
will
twenty
--
--
three
jo
s
journal
thirty
--


In [129]:
!grep -i "amy's valley" women.txt

There we have it - the counts from above were correct, and we could eliminate "t" and "s" from consideration with a `grep -v`, and we can further assume that the "ll" and "m" occurences are also from contractions, so we'll remove them as well.

In [132]:
!cat women.txt \
    | ./split.py \
    | ./lowercase.py \
    | ./stopwords.py \
    | grep -v -oE '^s|t|m|ll$' \
    | sort \
    | uniq -c \
    | sort -rn \
    | head -25

1362 jo
 725 one
 658 up
 598 laurie
 550 don
 492 very
 462 good
 399 now
 393 go
 380 old
 375 never
 355 over
 331 away
 319 know
 291 down
 279 young
 268 girls
 254 face
 252 asked
 245 herself
 242 day
 240 looked
 238 look
 218 long
 214 love


Here we have a final count.  It is interesting to note that these counts of character names (Jo, Meg, etc.) are slightly different from before, perhaps due to punctuation handling, but it seems beyond the scope of the question to answer it precisely.

## Extra credit - parallel stop words

### Use GNU parallel to count the 25 most common words across all the 109 texts in the zip file provided, with stop words removed.

In [134]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/texts.zip

--2016-09-26 15:26:10--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/projects/project-01/texts.zip
Resolving raw.githubusercontent.com... 151.101.32.133
Connecting to raw.githubusercontent.com|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12668137 (12M) [application/octet-stream]
Saving to: ‘texts.zip’

texts.zip           100%[===================>]  12.08M   894KB/s    in 11s     

2016-09-26 15:26:22 (1.08 MB/s) - ‘texts.zip’ saved [12668137/12668137]



In [135]:
!unzip -l texts.zip | head -5

Archive:  texts.zip
  Length     Date   Time    Name
 --------    ----   ----    ----
    52510  04-09-05 12:40   10001.txt
   306892  01-20-09 00:00   10002.txt


In [136]:
!mkdir all-texts

In [137]:
!unzip -d all-texts texts.zip

Archive:  texts.zip
  inflating: all-texts/10001.txt     
  inflating: all-texts/10002.txt     
  inflating: all-texts/10003.txt     
  inflating: all-texts/10004.txt     
  inflating: all-texts/10005.txt     
  inflating: all-texts/10006.txt     
  inflating: all-texts/10007.txt     
  inflating: all-texts/10008.txt     
  inflating: all-texts/10009.txt     
  inflating: all-texts/10010.txt     
  inflating: all-texts/10011.txt     
  inflating: all-texts/10012.txt     
  inflating: all-texts/10013.txt     
  inflating: all-texts/10014.txt     
  inflating: all-texts/10015.txt     
  inflating: all-texts/10016.txt     
  inflating: all-texts/10017.txt     
  inflating: all-texts/10018.txt     
  inflating: all-texts/10019.txt     
  inflating: all-texts/10020.txt     
  inflating: all-texts/10021.txt     
  inflating: all-texts/10023.txt     
  inflating: all-texts/10024.txt     
  inflating: all-texts/10025.txt     
  inflating: all-texts/10026.txt     
  inflating: all-texts/10027.t

In [178]:
!time ls all-texts/*.txt \
    | parallel --eta -j+0 "grep -oE '\w{1,}' {} | tr '[:upper:]' '[:lower:]' | grep -v -oE '^s|t|m|l|ll|d$' | ./stopwords.py >> all-words.txt"


Computers / CPU cores / Max jobs to run
1:local / 8 / 8

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 0 AVG: 0.04s  local:0/108/100%/0.0s 

real	0m3.924s
user	0m23.976s
sys	0m1.810s


In the above line, I've limited the word size to one character, removed common contractions, and piped the overall result through the new `stopwords.py` Python filter.

In [179]:
!wc -l all-words.txt

  831140 all-words.txt


In [180]:
!time sort all-words.txt | uniq -c | sort -rn | head -25

19589 one
12235 up
9508 very
9503 now
8539 upon
7101 over
6892 down
6749 day
6671 work
6670 know
6270 before
6107 here
5874 1
5666 never
5591 way
5466 go
4760 even
4689 new
4614 back
4312 again
4310 away
4264 being
3991 don
3949 under
3903 house
sort: write failed: standard output: Broken pipe
sort: write error

real	0m8.251s
user	0m8.268s
sys	0m0.037s


Above we have the top 25 words across all 109 texts, with common English stop words removed.  It is easy to imagine that a broader set of stop words would remove many more common words like "one", "up", "down", and "here", but that is the tradeoff of choosing any single list.